# Recuperação de Processos
É descrito aqui o modelo de recuperação dos processos e seus dados para população do banco.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import requests, os, re, json
from PyPDF2 import PdfReader
from time import sleep
from unidecode import unidecode
from pyreadr import read_r
from datetime import datetime

## Coleta de nomes de advogados
Para obter os processos, é necessária uma chave de busca nos portais de consulta processual do país. Houve uma tentativa inicial, tentando gerar de forma aleatória várias códigos CNJ, mas essa implementação se mostrou custosa e pouco eficiente. 

Observando os processos, foi fácil ver que o ponto comum entre todos os processos é a presença de um advogado ou promotor de justiça. Dessa forma, o nome dos advogados se tornou chave para busca de processos, tornando necessária a listagem de advogados por cada estado.

Assim, iniciou o processo de coleta dos nomes dos advogados a partir dos resultados dos exames da OAB.

In [3]:
driver = webdriver.Firefox()
driver.get("https://oab.fgv.br/")

Inicialmente, os links para a página dos resultados são listados e acessados sequencialmente, seguido do download do último arquivo PDF disponível.

In [5]:
links = driver.find_elements(By.CSS_SELECTOR, "ul li a")
links = [link.get_attribute('href') for link in links]

for link in links:
    driver.get(link)

    select = Select(driver.find_element(By.CSS_SELECTOR, "select"))
    select.select_by_visible_text('OAB / RJ')

    files_list = driver.find_elements(By.CSS_SELECTOR, "table tr a")

    if len(files_list) == 0:
        continue

    files_list = [file.get_attribute("href") for file in files_list]
    files_list = [file for file in files_list if file.endswith('.pdf')]

    filename = files_list[0].split('/')[-1]
    response = requests.get(files_list[0])
    open(f"oab_pdfs/{filename}", "wb").write(response.content)

Após o download dos arquivos em PDF, os textos são extraídos tratados e separados utilizando REGEX e alocados em um dicionário que separa os advogados por estado.

In [87]:
pdf_path = "./oab_pdfs/"
pdf_list = os.listdir(pdf_path)

names = {}

for pdf_file in pdf_list:
    pdf_obj = open(pdf_path+pdf_file, 'rb')
    pdf_reader = PdfReader(pdf_obj)

    all_text = ""
    for i in range(pdf_reader.numPages):
        page_obj = pdf_reader.getPage(i)
        text = page_obj.extractText()
        all_text += (text)
        if all_text[-1] == ' ':
            all_text = all_text[:-2]

    all_text = all_text.replace("\n", "")
    state_marks = re.findall("\d*. OAB / \w{2}", all_text)
    state_marks = [re.sub("\d*. OAB /", "", state) for state in state_marks]
    
    divisions = re.split("\d*. OAB / \w{2}", all_text)
    for i, division in enumerate(divisions):
        division = re.sub("\d.\d. [A-Z] ", "", division)
        division = division.split("/")
        for l, item in enumerate(division):
            item = item.split(",")[-1]
            item = item.replace(".", "")
            item = item.replace("  ", " ")
            division[l] = item.strip()
        divisions[i] = division

    del divisions[0]

    for i, state in enumerate(state_marks):
        state = state.strip()
        if state not in names:
            names[state] = []
        names[state] += divisions[i].replace("  ", " ")

Como é possível ver, o resgaste dos nomes dos advogados é bastante demorado. Por isso, o dicionário foi exportado para um arquivo json, para reduzir tempo de futuras coletas.

In [92]:
with open("names.json", "w") as output_file:
    json.dump(names, output_file)

Após a recuperação dos dados do arquivo JSON, incia-se a recuperação dos processos

In [2]:
with open("names.json") as file:
    names = json.load(file)

## Coleta dos processos
Agora, os processos são coletados sequencialmente a partir de buscas no site do PJE do Ceará. São realizadas buscas pelo nome do advogado, e o resultados são coletados e armazenados em um lista de dicionários. Esses dicionários registram:
- Assunto do processo
- Classe judicial
- Data
- Jurisdição
- Lista de movimentações dos processos
- Número do processo
- Órgão julgador
- Participantes do Polo Ativo
- Participantes do Polo Passivo
- Tribunal

In [50]:
pje_links = {
    "AM": {
        "link": "https://pje1g.trf1.jus.br/consultapublica/ConsultaPublica/listView.seam",
        "base_path": "https://pje1g.trf1.jus.br",
        "tribunal": "TRF1"
    },
    "AP": {
        "link": "https://pje.tjap.jus.br/1gconsulta/ConsultaPublica/listView.seam",
        "base_path": "https://pje.tjap.jus.br",
        "tribunal": "TJPA"
    },
    "CE": {
        "link": "https://pje.tjce.jus.br/pje1grau/ConsultaPublica/listView.seam",
        "base_path": "https://pje.tjce.jus.br",
        "tribunal": "TJCE"
    },
    "DF": {
        "link": "https://www.cnj.jus.br/pjecnj/ConsultaPublica/listView.seam",
        "base_path": "https://www.cnj.jus.br",
        "tribunal": "CNJ"
    },
    "ES": {
        "link": "https://sistemas.tjes.jus.br/pje/ConsultaPublica/listView.seam",
        "base_path": "https://sistemas.tjes.jus.br",
        "tribunal": "TJES"
    },
    "MG": {
        "link": "https://pje-consulta-publica.tjmg.jus.br/",
        "base_path": "https://pje-consulta-publica.tjmg.jus.br/",
        "tribunal": "TJMG"
    },
    "MT": {
        "link": "https://pje.tjmt.jus.br/pje/ConsultaPublica/listView.seam",
        "base_path": "https://pje.tjmt.jus.br",
        "tribunal": "TJMT"
    },
    "PB": {
        "link": "https://consultapublica.tjpb.jus.br/pje/ConsultaPublica/listView.seam",
        "base_path": "https://consultapublica.tjpb.jus.br",
        "tribunal": "TJPB"
    },
    "PI": {
        "link": "https://pje1g.trf1.jus.br/consultapublica/ConsultaPublica/listView.seam",
        "base_path": "https://pje1g.trf1.jus.br",
        "tribunal": "TRF1"
    },
    "TO": {
        "link": "https://pje1g.trf1.jus.br/consultapublica/ConsultaPublica/listView.seam",
        "base_path": "https://pje1g.trf1.jus.br",
        "tribunal": "TRF1"
    }
}

In [ ]:
driver = webdriver.Firefox()

def parse_td(td):
    title = td.find_element(By.CLASS_NAME, "name")
    title = title.text.strip()
    content = td.find_element(By.CLASS_NAME, "value")
    content = content.text.strip() 
    return (title, content)

def parse_tr(tr):
    tds = tr.find_elements(By.TAG_NAME, "td")
    nome = tds[0]
    nome = nome.text.strip()
    status = tds[1]
    status = status.text.strip()
    return {'nome': nome, 'status': status}
    
for state, data in pje_links.items():
    processos = []
    link, base_path, tribunal = data.values()

    try:
        for name in names[state][1000:0:-1]:
            driver.get(link)
            input = driver.find_elements(By.CSS_SELECTOR, ".rich-panel-body .propertyView")
            input = [field for field in input if "Nome do advogado" in field.text]
            input = input[0].find_element(By.TAG_NAME, "input")
            submit_btn = driver.find_element(By.CSS_SELECTOR, ".rich-panel-body input[type=button]")

            input.clear()
            input.send_keys(name)
            submit_btn.click()

            try:
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, 'rich-mpnl-content')))
            except:
                print("skipped ", name)
                continue

            table = driver.find_element(By.ID, "fPP:processosTable:tb")
            results = table.find_elements(By.CSS_SELECTOR, "tr td a")

            if len(results) == 0:
                continue

            results = [result.get_attribute('onclick') for result in results]
            results = [result[:-2].replace("openPopUp('Consulta pública','", "") for result in results]
            results = list(dict.fromkeys(results))

            for result in results:
                processo = {
                    'assunto' : None,
                    'classe_judicial': None,
                    'data': None,
                    'jurisdicao': None,
                    'movimentacao_do_processo': [],
                    'numero': None,
                    'orgao_julgador': None,
                    'outros_interessados': [],
                    'polo_ativo': [],
                    'polo_passivo': [],
                    'tribunal': tribunal,
                }

                driver.get(base_path + result)
                general_data = driver.find_element(By.CLASS_NAME, "rich-stglpanel")
                general_data = general_data.find_elements(By.TAG_NAME, "td")
                general_data = dict([parse_td(td) for td in general_data])
                
                processo['numero'] = general_data['Número Processo'].strip()
                processo['data'] = general_data['Data da Distribuição'].strip()
                processo['jurisdicao'] = general_data['Jurisdição'].strip()
                processo['classe_judicial'] = general_data['Classe Judicial'].strip()
                processo['assunto'] = general_data['Assunto'].strip()
                processo['orgao_julgador'] = general_data['Órgão Julgador'].strip()

                panels = driver.find_elements(By.CLASS_NAME, 'rich-panel')
                for panel in panels:
                    key = panel.find_element(By.CLASS_NAME, 'rich-panel-header')
                    key = key.text.lower()
                    key = unidecode(key).replace(' ', '_')
                    if key not in processo:
                        continue
                    if key == 'movimentacao':
                        content = panel.find_elements(By.CSS_SELECTOR, "tbody td:nth-child(1)")
                        content = [td.text.strip() for td in content if len(td.text.strip()) > 0]
                        content = [mov.split('-') for mov in content if len(mov.split('-')) == 2]
                    else:
                        content = panel.find_elements(By.CSS_SELECTOR, "tbody tr.rich-table-row")
                        content = [parse_tr(tr) for tr in content]
                    processo[key] = content

                processos.append(processo)
    except:
        print("Falha no estado ", state)
        print(f"{len(processos)} processos registrados no estado {state}")
        if len(processos) > 0:
            with open(f"./states/{state}.json", "w") as output_file:
                json.dump(processos, output_file)
        continue

    print(f"{len(processos)} processos registrados no estado {state}")
    if len(processos) > 0:
        with open(f"./states/{state}.json", "w") as output_file:
            json.dump(processos, output_file)


Após a obtenção dos processos, eles são unificados em um único arquivo JSON

In [5]:
processos = []

states_folder = './states'
states_folder = os.listdir(states_folder)
for file in states_folder:
    with open('./states/' + file) as fl:
        processos += json.load(fl)

with open("processos.json", "w") as output_file:
    json.dump(processos, output_file)

## Processamento dos dados
Os dados, após a coleta, são processados para separar as entidades dos dados, formatar datas e tipos.

In [6]:
for i, processo in enumerate(processos):
    polo_ativo = processo['polo_ativo']
    polo_passivo = processo['polo_passivo']

    def process_polo(polo):
        for i, participante in enumerate(polo):
            participante_dict = {}
            nome, ativo = participante.values()
            tipo_participante = re.findall('\(.*\)', nome)
            if tipo_participante:
                tipo_participante = tipo_participante[0]
                tipo_participante = tipo_participante.replace('(', '')
                tipo_participante = tipo_participante.replace(')', '')
                tipo_participante = tipo_participante.lower()
                nome = re.sub('\(.*\)', '', nome)
            cpfs = re.findall('CPF: \d{3}.\d{3}.\d{3}-\d{2}', nome)
            cnpjs = re.findall('CNPJ: \d{2}.\d{3}.\d{3}/\d{4}-\d{2}', nome)
            for item in cpfs + cnpjs:
                old_item = str(item)
                item = item.replace('.', '')
                item = item.replace('-', '')
                item = item.replace('/', '')
                item = item.replace('CNPJ: ', '')
                item = item.replace('CPF: ', '')
                nome = nome.replace(old_item, item)
            nome = nome.split('-')
            nome = [n.strip() for n in nome]
            participante_dict['tipo_participante'] = tipo_participante
            participante_dict['ativo'] = ativo == 'Ativo'
            participante_dict['nome'] = nome[0]
            participante_dict['OAB'] = None
            participante_dict['text'] = None
            if tipo_participante == 'advogado':
                participante_dict['id'] = nome[-1]
                participante_dict['OAB'] = nome[1]
            elif tipo_participante == 'exequente':
                participante_dict['id'] = nome[-1]
            elif tipo_participante == 'autora':
                participante_dict['id'] = nome[1].replace('CPF: ', '')
            
            polo[i] = participante_dict
        return polo
    
    polo_ativo = process_polo(polo_ativo)
    polo_passivo = process_polo(polo_passivo)

    data = processo['data'].strip()
    data = [int(item) for item in data.split('/')]
    data = datetime(day=data[0], month=data[1], year=data[2])
    data = data.isoformat()

    numero = str(processo['numero'])
    numero = numero.replace('.','')
    numero = numero.replace('-', '')

    processos[i]['numero'] = numero
    processos[i]['polo_ativo'] = polo_ativo
    processos[i]['polo_passivo'] = polo_passivo
    processos[i]['tribunal_sigla'] = processos[i].pop('tribunal')
    processos[i]['data'] = data
    processos[i]['uf'] = None
    if processo['tribunal_sigla'].startswith('TJ') and len(processo['tribunal_sigla']) == 4:
        processos[i]['uf'] = processo['tribunal_sigla'][2:]



Por fim, os dados processados são organizados e alocados em um arquivo JSON

In [7]:
processos = sorted(processos, key=lambda x: x['numero'])
with open(f"database.json", "w") as input_file:
    json.dump(processos, input_file)